### Import Library

In [38]:
import pandas 
import os
import pandas as pd
import faiss
from uuid import uuid4
from langchain_core.documents import Document
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_community.utilities import SQLDatabase
%load_ext autoreload
%autoreload 2
from dotenv import load_dotenv
load_dotenv(r'C:\Users\VR679RX\OneDrive - EY\Documents\Projects\Projects - 2024\LangGraph_Academy\gpt4o.env')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [25]:
db = SQLDatabase.from_uri("sqlite:///xinfinity_agent.db")

In [26]:
# Print the dialect and available tables
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['billing_table', 'customer_service_table', 'customer_table']


In [27]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [43]:
llm = AzureChatOpenAI(temperature=0,
                           api_key=os.getenv('AZURE_OPENAI_API_KEY'),
                           azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                           openai_api_version=os.getenv('AZURE_OPENAI_VERSION'),
                           azure_deployment=os.getenv('AZURE_GPT4O_MODEL')
                           )

embeddings = AzureOpenAIEmbeddings(
                            api_key=os.getenv('AZURE_OPENAI_API_KEY'),
                            azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                            azure_deployment=os.getenv('AZURE_OPENAI_EMBEDDINGS_MODEL'),
                            openai_api_version=os.getenv('AZURE_OPENAI_VERSION'),
                            chunk_size = 1
                            )

### ReAct SQL Agent

In [60]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000021271FA3A10>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000021271FA3A10>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000021271FA3A10>),
 QuerySQLCheckerTool(description='Use this tool to 

In [61]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
prompt_template.messages[0].prompt.template = """You are an intelligent assistant helping users explore a customer billing system.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\n\nTo start you should ALWAYS look at the tables in the database to see what you can query.\nDo NOT skip this step.\nThen you should query the schema of the most relevant tables. \nBefore executing a query, make sure you have:
1. The correct **table** to query.
2. All **mandatory fields** for that table.
3. Clear disambiguation if needed (e.g., same name, multiple services).

It is important that You ask follow-up questions to gather missing fields based on this schema:
- `customer_table` needs customer_id.
- `customer_service_table` needs customer_id + service_id.
- `billing_table` needs customer_id + service_id (+ optional billing_date, status, plan_name).

When ready, construct a SQL-like query or structured representation to fetch the data."""
prompt_template.messages[0].pretty_print()

================================ System Message ================================

You are an intelligent assistant helping users explore a customer billing system.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, D

In [62]:
system_message = prompt_template.format(dialect="SQLite", top_k=10)

In [63]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

sql_agent_executor = create_react_agent(model= llm, 
                                        tools= tools, 
                                        name= 'sql_agent_executor',
                                        prompt= system_message)

In [64]:
# question = "How much did I spend in the last 6 months?"

# for step in sql_agent_executor.stream(
#     {"messages": [{"role": "user", "content": question}]},
#     stream_mode="values",
# ):
#     step["messages"][-1].pretty_print()

### Create RAG Agent

In [65]:
from langchain.tools.retriever import create_retriever_tool

In [66]:
# from DataProcessing.DocumentParser import DocumentParser
# from IndexGeneration.DocumentChunk import DocumentChunk

# parser = DocumentParser()
# chunker = DocumentChunk()

# dataframe = pd.DataFrame()
# for pdf in ['HOW5220_DG558_XB6_getting_started.pdf','RSA_01012025.pdf']:    
#     result = parser.extract_text(pdf,type='local')
#     chunk_df = chunker.process_chunk(pd.DataFrame(result),1500)
#     chunk_df['source'] = pdf
#     dataframe = pd.concat([chunk_df,dataframe])

# dataframe = dataframe.reset_index(drop=True)

# documents = []
# for _,row in dataframe.iterrows():
#     documents.append(
#         Document(page_content=row['merged_content'],
#                  metadata={
#                      'source':row['source'],
#                      'page_range':row['page_range'],
#                      'chunk_coordinates':row['chunk_coordinates']
#                  })
#     )

# index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
# vector_store = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={},
# )
# uuids = [str(uuid4()) for _ in range(len(documents))]
# vector_store.add_documents(documents=documents, ids=uuids)
# vector_store.save_local('vector_store_index')

In [67]:
vector_store = FAISS.load_local(
    "vector_store_index", embeddings, allow_dangerous_deserialization=True
)


In [68]:
retriever_tool = create_retriever_tool(
    retriever= vector_store.as_retriever(),
    name='retriever_tool',
    description='For fetching any company related policies or guidelines information'
)

In [90]:
from langchain_core.prompts import ChatPromptTemplate

system_message = ChatPromptTemplate([
    ("system", "You are helpful assitant. You need to help user to answer query related to company's policies guidelines"),
     "{messages}"
])

In [91]:
rag_react_agent = create_react_agent(model= llm, 
                                        tools= [retriever_tool], 
                                        name= 'rag_agent_executor',
                                        prompt= system_message)
#agent_executor = AgentExecutor(agent=rag_react_agent, tools=tools)

In [92]:
question = 'what is liability for the damage?'
rag_react_agent.invoke({"messages": [{"role": "user", "content": question}]})

{'messages': [HumanMessage(content='what is liability for the damage?', additional_kwargs={}, response_metadata={}, id='18c34467-b1e6-4398-8850-91420468897e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_X71Nqp1myQwOnNUBC2tZk70Q', 'function': {'arguments': '{"query":"liability for the damage"}', 'name': 'retriever_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 118, 'total_tokens': 139, 'completion_tokens_details': {'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_65792305e4', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'fi

## Supervisor Agent

In [94]:
from langgraph_supervisor import create_supervisor

In [95]:
workflow = create_supervisor(
    [sql_agent_executor, rag_react_agent],
    model=llm,
    prompt=(
        "You are a team supervisor managing a customer billing sql analysis expert and a math expert. "
        "For customer billing realted questions, use sql_agent_executor. "
        "For queries related to company's policies and guidelines, use rag_agent_executor."
        "Avoid mentioning any agent name in your response"
    )
)

In [96]:
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": 'what is liability for the damage?'
        }
    ]
})

In [97]:
result

{'messages': [HumanMessage(content='what is liability for the damage?', additional_kwargs={}, response_metadata={}, id='62ee40e7-d302-4d7f-94fb-f06a509aaa76'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JUK8uiiqeJKPiW9VekytRmDR', 'function': {'arguments': '{}', 'name': 'transfer_to_rag_agent_executor'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 117, 'total_tokens': 132, 'completion_tokens_details': {'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_65792305e4', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 's